In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr" #"all"

In [2]:
import gc
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import dscribe
import ase

bond_types = ['1JHC', '1JHN', '2JHC', '2JHH', '2JHN', '3JHC', '3JHH', '3JHN']
pickles = [i for i in os.listdir('data') if ".pkl" in i]
#dfs = {i[:-4] : pd.read_pickle('data/'+i) for i in pickles}
#print ("\t".join(dfs.keys()))
print (pickles)
more_pickles = [i for i in os.listdir('data/additional') if ".pkl" in i]
print (more_pickles)
len_train = 4658147
def read_pkl(fname):
    return pd.read_pickle('data/' + fname + '.pkl')

['traintestadj.pkl', 'distances.pkl', 'magnetic_shielding_tensors.pkl', 'traintestpathfixed.pkl', 'df_graph_fixed.pkl', 'test.pkl', 'traintest.pkl', 'dipole_moments.pkl', 'structures_reindexed.pkl', 'train_molecule_names.pkl', 'lengthes_4.pkl', 'train_contrib.pkl', 'df_graph.pkl', 'traintestpathmulti.pkl', 'traintestall.pkl', 'train.pkl', 'potential_energy.pkl', 'struc_and_dist.pkl', 'structures.pkl', 'strucs_some_feat.pkl', 'traintestpath.pkl', 'test_molecule_names.pkl', 'dist_df.pkl', 'mulliken_charges.pkl', 'traintestadjfixed.pkl']
['aseAtoms.pkl', 'ASCF_features.pkl', 'ASCF_features1208.pkl', 'strucs_nn_features.pkl', 'nn_atoms_mult_total.pkl']


In [3]:
import torch
from importlib import reload
from torch import nn
from torch.functional import F
from sklearn.model_selection import train_test_split
from torch.optim import Adam, SGD
torch.manual_seed(999)
import tqdm

In [8]:
gc.collect()

0

In [5]:
device = torch.device('cuda:0')

In [6]:
features = pd.read_pickle('features/nn_features1708.pkl')
df_graph = pd.read_pickle('data/df_graph_fixed.pkl')
strucs = pd.read_pickle('data/struc_and_dist.pkl')
strucs_nn_features = pd.read_pickle('data/additional/strucs_nn_features.pkl')
molecule_names = pd.read_pickle('data/train_molecule_names.pkl').unique()
trantest = pd.read_pickle('cc/traintestnnindexed.pkl')
weights = pd.read_pickle('cc/weights.pkl')
coupling_means = pd.read_pickle('cc/coupling_constant_means.pkl')
coupling_stds = pd.read_pickle('cc/coupling_constant_stds.pkl')

In [7]:
train_mols, val_mols = train_test_split(molecule_names, test_size=0.1, random_state=42)

In [9]:
train_mols

array(['dsgdb9nsd_031591', 'dsgdb9nsd_039358', 'dsgdb9nsd_023449', ...,
       'dsgdb9nsd_120373', 'dsgdb9nsd_001459', 'dsgdb9nsd_024846'],
      dtype=object)

In [10]:
checkpoint = torch.load('/home/ikari/savednn/epoch-59.pt')

In [11]:
net.load_state_dict(checkpoint['net_state_dict'])
poolnet[2].load_state_dict(checkpoint['poolnet_2_state_dict'])
poolnet[3].load_state_dict(checkpoint['poolnet_3_state_dict'])
poolnet[4].load_state_dict(checkpoint['poolnet_4_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [11]:
torch.save({
            'epoch': -1,
            'net_state_dict': net.state_dict(),
            'poolnet_2_state_dict' : poolnet[2].state_dict(),
            'poolnet_3_state_dict' : poolnet[3].state_dict(),
            'poolnet_4_state_dict' : poolnet[4].state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            #'loss': loss,
            }, '/home/ikari/savednn/epoch-minus2.pt')

In [14]:
trantest.tail()

atom_index_0  atom_index_1  type  \
molecule_name    id                                          
dsgdb9nsd_133885 7163684            15             3  2JHC   
                 7163685            15             4  2JHC   
                 7163686            15             6  3JHC   
                 7163687            15             7  2JHC   
                 7163688            15             8  1JHC   

                          scalar_coupling_constant           path  \
molecule_name    id                                                 
dsgdb9nsd_133885 7163684                       NaN     [15, 8, 3]   
                 7163685                       NaN     [15, 8, 4]   
                 7163686                       NaN  [15, 8, 7, 6]   
                 7163687                       NaN     [15, 8, 7]   
                 7163688                       NaN        [15, 8]   

                                pathmulti  
molecule_name    id                        
dsgdb9nsd_133885 7163684     [[15, 8, 3]]  
                 7163685     [[15, 8, 4]]  
                 7163686  [[15, 8, 7, 6]]  
                 7163687     [[15, 8, 7]]  
                 7163688        [[15, 8]]

In [15]:
real_answ = pd.DataFrame(data=trantest[['type', 'scalar_coupling_constant']].values,
                         index = trantest.index.get_level_values(1),
                         columns = ['type', 'scalar_coupling_constant'])
real_answ['predicted_answ'] = 0.
real_answ['is_validation'] = False

In [16]:
real_answ.head()

,type,scalar_coupling_constant,predicted_answ,is_validation
id,,,,
0,1JHC,-0.556351,0.0,False
1,2JHH,-0.243842,0.0,False
2,2JHH,-0.243289,0.0,False
3,2JHH,-0.243163,0.0,False
4,1JHC,-0.556362,0.0,False


In [10]:
import time

In [11]:
import pickle
with open('/home/ikari/savednn/all_all_np_protocol4.pkl', 'rb') as f:
    all_all = pickle.load(f)

In [16]:
#all_all['all_xyz']

{'train': tensor([[[ 2.2728e-03,  1.5551e+00, -7.2391e-02],
          [ 1.5667e+00,  1.4931e+00,  4.8571e-02],
          [ 1.3709e+00,  9.3969e-02,  5.9057e-01],
          ...,
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00]],
 
         [[-3.5354e-03,  1.3420e+00,  4.5979e-02],
          [-1.4895e-04, -8.4822e-02, -5.5287e-02],
          [ 6.0981e-01, -5.7608e-01, -1.1281e+00],
          ...,
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00]],
 
         [[-2.8357e-01,  3.1425e-02, -2.4104e-01],
          [ 1.7340e-02,  1.1609e+00,  4.1810e-02],
          [ 1.3741e+00,  1.7809e+00, -2.1988e-01],
          ...,
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00]],
 
         ...,
 
   

In [17]:
#torch.cuda.empty_cache()
'''all_xyz = {'train':[], 'validate':[]}
all_ytrue = {'train':[], 'validate':[]}
all_e_features = {'train':[], 'validate':[]}
all_A = {'train':[], 'validate':[]}

all_ytrue = {'train':[], 'validate':[]}
all_batch_type = {'train':[], 'validate':[]}
all_batch_weights = {'train':[], 'validate':[]}
all_tt = {'train':[], 'validate':[]}
all_ff =  {'train':[], 'validate':[]}

#all_tensor_weights = {'train':{2:[],3:[],4:[]}, 'validate':[]}
indices_for_LLen = {'train':{2:[],3:[],4:[]}, 'validate':{2:[],3:[],4:[]}} '''

"all_xyz = {'train':[], 'validate':[]}\nall_ytrue = {'train':[], 'validate':[]}\nall_e_features = {'train':[], 'validate':[]}\nall_A = {'train':[], 'validate':[]}\n\nall_ytrue = {'train':[], 'validate':[]}\nall_batch_type = {'train':[], 'validate':[]}\nall_batch_weights = {'train':[], 'validate':[]}\nall_tt = {'train':[], 'validate':[]}\nall_ff =  {'train':[], 'validate':[]}\n\n#all_tensor_weights = {'train':{2:[],3:[],4:[]}, 'validate':[]}\nindices_for_LLen = {'train':{2:[],3:[],4:[]}, 'validate':{2:[],3:[],4:[]}} "

In [84]:
all_all['all_tt']['train'][0]

array([list([8, 0]), list([8, 0, 1]), list([8, 0, 1, 2]),
       list([8, 0, 7, 6]), list([8, 0, 7]), list([8, 0, 9]),
       list([8, 0, 1, 10]), list([8, 0, 1, 11]), list([8, 0, 7, 17]),
       list([9, 0]), list([9, 0, 1]), list([9, 0, 1, 2]),
       list([9, 0, 7, 6]), list([9, 0, 7]), list([9, 0, 1, 10]),
       list([9, 0, 1, 11]), list([9, 0, 7, 17]), list([10, 1, 0]),
       list([10, 1]), list([10, 1, 2]), list([10, 1, 2, 3]),
       list([10, 1, 0, 7]), list([10, 1, 11]), list([11, 1, 0]),
       list([11, 1]), list([11, 1, 2]), list([11, 1, 2, 3]),
       list([11, 1, 0, 7]), list([12, 3, 2, 1]), list([12, 3, 2]),
       list([12, 3]), list([12, 3, 4]), list([12, 3, 2, 7]),
       list([12, 3, 4, 13]), list([12, 3, 4, 14]), list([13, 4, 3, 2]),
       list([13, 4, 3]), list([13, 4]), list([13, 4, 5, 6]),
       list([13, 4, 14]), list([14, 4, 3, 2]), list([14, 4, 3]),
       list([14, 4]), list([14, 4, 5, 6]), list([15, 6, 7, 0]),
       list([15, 6, 7, 2]), list([15, 6, 5, 

In [37]:
all_all['all_batch_weights']['train']

array([tensor([0.2542, 0.1580, 0.1193, 0.1193, 0.1580, 0.4767, 0.3051, 0.3051, 0.3051,
        0.2542, 0.1580, 0.1193, 0.1193, 0.1580, 0.3051, 0.3051, 0.3051, 0.1580,
        0.2542, 0.1580, 0.1193, 0.1193, 0.4767, 0.1580, 0.2542, 0.1580, 0.1193,
        0.1193, 0.1193, 0.1580, 0.2542, 0.1580, 0.1193, 0.3051, 0.3051, 0.1193,
        0.1580, 0.2542, 0.1193, 0.4767, 0.1193, 0.1580, 0.2542, 0.1193, 0.1193,
        0.1193, 0.1193, 0.2542, 0.1580, 0.4767, 0.3051, 0.1193, 0.1193, 0.1193,
        0.2542, 0.1580, 0.3051, 0.1580, 0.1193, 0.1580, 0.1193, 0.1580, 0.2542]),
       tensor([0.2542, 0.1193, 0.4767, 0.4767, 0.2542, 0.1193, 0.4767, 0.2542, 0.1193,
        0.1193, 1.5083, 0.2542, 0.1580, 0.1193, 0.1193, 1.0778, 0.4767, 0.1193,
        1.5083, 0.2542, 0.1580, 0.1193, 0.1193, 1.0778, 0.1193, 0.1580, 0.2542,
        0.1193, 1.0778, 0.4767, 0.4767, 0.1193, 0.1580, 0.2542, 0.1193, 1.0778,
        0.4767, 0.1193, 0.1580, 0.2542, 0.1193, 1.0778, 0.1193, 0.1193, 0.1580,
        0.1193, 0.2542, 

In [36]:
all_all['all_ytrue']['train']

array([tensor([-0.2297, -0.3029, -0.7453,  0.5372, -0.2281,  0.3803,  0.7131,  0.4949,
         0.4214, -0.5337, -0.1315,  0.7549, -0.5522, -0.1472,  0.7703, -0.9783,
         0.5661, -0.7900, -0.1862, -1.2661,  0.1441, -0.8732, -0.4279, -0.2038,
        -0.4755, -0.7037, -0.1919,  1.6076,  0.8926, -0.3233,  1.3633,  2.3689,
         1.0004, -0.7292, -0.5706,  0.2303, -0.8570, -0.4322,  0.5720, -1.2450,
         1.0497, -0.3842, -0.2745, -1.0844, -1.2085,  0.0655,  0.9738, -0.6204,
        -0.0227,  0.4489,  0.2093, -0.5963, -0.9178, -0.2437, -0.0538,  0.4742,
         0.6964, -0.3947, -0.8429, -0.9677,  0.0133, -0.2270, -0.6778]),
       tensor([ 2.2095e-01, -8.9543e-01,  4.2191e-02,  5.4897e-01,  2.5604e-01,
        -8.4400e-01,  5.1513e-01, -7.1779e-01,  1.4547e+00,  5.2463e-01,
        -5.4659e-01, -2.2214e-01, -3.7107e-04,  5.1068e-01, -1.6685e-01,
        -7.4367e-01, -7.1944e-01,  1.0822e+00, -3.1089e-01, -3.8985e-01,
         3.4125e-01, -7.1370e-01, -4.5322e-01, -7.5646e-01, -

array([7, 5, 4, 0, 4, 6, 2, 3, 9, 6, 2, 9, 1, 0, 7, 3, 3, 4, 1, 5, 2, 1,
       4, 6, 7, 2, 4, 9, 3, 4, 4, 2, 6, 5, 6, 2, 4, 3, 6, 2, 1, 8, 6, 4,
       1, 1, 6, 5, 5, 3, 6, 6, 5, 7, 5, 1, 0, 5, 5, 3, 2, 7, 2, 8, 9])

In [108]:
%%time
#lll = all_all['indices_for_LLen']['train'][4][1:128]

torch.cat(  [all_all['all_batch_weights']['train'][1:128][np.random.randint(10, size=65)][x] for x in np.random.randint(10, size=65)]).shape

CPU times: user 1.4 ms, sys: 787 µs, total: 2.19 ms
Wall time: 1.41 ms


torch.Size([3754])

In [118]:
%%time
#tmp = all_all['all_batch_weights']['train'][1:128][[0,0,1]]
torch.stack([x[y] for x,y in zip(all_all['all_batch_weights']['train'][1:128][np.random.randint(10, size=65)],   np.random.randint(10, size=65))])


zzzz = np.array([x[y] for x,y in zip(all_all['all_tt']['train'][1:128][np.random.randint(10, size=65)],   np.random.randint(10, size=65))] , dtype=object)

CPU times: user 1.14 ms, sys: 0 ns, total: 1.14 ms
Wall time: 869 µs


In [122]:
zzzz

array([list([10, 0]), list([9, 0, 1, 12]), list([9, 0, 1]),
       list([9, 0, 1, 10]), list([9, 0, 1]), list([9, 0, 10]),
       list([10, 2, 3]), list([9, 0, 1]), list([8, 0]),
       list([10, 1, 2, 8]), list([9, 0, 4]), list([9, 0]), list([9, 0]),
       list([9, 0, 11]), list([9, 0, 4]), list([9, 0, 1, 2]),
       list([10, 0, 1, 2]), list([9, 0, 1, 12]), list([9, 0, 10]),
       list([9, 0, 11]), list([10, 0]), list([10, 1, 2, 11]),
       list([10, 1, 2, 8]), list([9, 0, 4]), list([10, 0, 1]),
       list([10, 0, 1]), list([10, 0, 1, 2]), list([10, 0, 1]),
       list([9, 1, 2, 10]), list([10, 0, 1, 2]), list([10, 0, 11]),
       list([9, 0, 10]), list([10, 0, 1]), list([10, 0, 1, 2]),
       list([8, 0, 9]), list([9, 0, 11]), list([9, 0, 1, 2]),
       list([10, 0]), list([10, 0, 1, 2]), list([10, 0, 1]),
       list([10, 0, 1]), list([8, 0, 9]), list([11, 2, 1, 0]),
       list([9, 0, 11]), list([9, 0, 1, 2]), list([9, 0, 10]),
       list([9, 0, 4]), list([10, 0, 1]), list([1

In [114]:
all_all['all_batch_weights']['train'][1:128][[0,0,1]][0][0]

tensor(0.2542)

In [95]:
%%time
np.concatenate([np.repeat(x, len(lll[x][0])) for x in range(len(lll))])

CPU times: user 2.97 ms, sys: 0 ns, total: 2.97 ms
Wall time: 2.26 ms


array([  0,   0,   0, ..., 126, 126, 126])

In [96]:
%%time
np.concatenate([x[0] for x in lll])


CPU times: user 1.2 ms, sys: 677 µs, total: 1.87 ms
Wall time: 1.08 ms


array([ 1,  5,  8, ..., 15, 17, 21])

In [97]:
all_all['all_batch_weights']['train'][1:4]

array([tensor([0.2542, 0.1193, 0.4767, 0.4767, 0.2542, 0.1193, 0.4767, 0.2542, 0.1193,
        0.1193, 1.5083, 0.2542, 0.1580, 0.1193, 0.1193, 1.0778, 0.4767, 0.1193,
        1.5083, 0.2542, 0.1580, 0.1193, 0.1193, 1.0778, 0.1193, 0.1580, 0.2542,
        0.1193, 1.0778, 0.4767, 0.4767, 0.1193, 0.1580, 0.2542, 0.1193, 1.0778,
        0.4767, 0.1193, 0.1580, 0.2542, 0.1193, 1.0778, 0.1193, 0.1193, 0.1580,
        0.1193, 0.2542, 1.5083, 0.4767, 0.1193, 0.1193, 0.1580, 0.1193, 0.2542,
        1.5083]),
       tensor([0.2542, 0.1580, 0.1193, 0.3051, 0.3051, 0.1580, 0.2542, 0.1580, 1.0778,
        0.4767, 0.3051, 0.3051, 0.1580, 0.2542, 0.1580, 1.0778, 0.3051, 0.3051,
        0.1193, 0.1580, 0.2542, 1.5083, 0.1193, 1.0778, 0.4767, 0.1193, 0.1580,
        0.2542, 1.5083, 0.1193, 1.0778, 0.1193, 1.5083, 0.2542, 0.1580, 1.0778,
        1.0778, 0.3051, 1.0778, 0.1580, 0.2542, 1.5083, 1.0778]),
       tensor([0.2542, 0.1580, 0.1193, 0.1193, 0.1193, 0.4767, 0.4767, 0.2542, 0.1580,
        0.1193,

In [72]:
all_all['all_batch_weights']['train'][1:4][1][lll[1][0]]

tensor([0.1193, 0.3051, 0.3051, 1.0778, 0.3051, 0.3051, 1.0778, 0.3051, 0.3051,
        0.1193, 0.1193, 1.0778, 0.1193, 0.1193, 1.0778, 0.1193, 1.0778, 1.0778,
        0.3051, 1.0778, 1.0778])

In [ ]:
np.apply_along_axis(lambda x: x[lll])

In [59]:
lll
zzz = np.array([np.array([ 0,  4,  7, 11, 19, 26, 33, 39, 46, 53]), np.array([ 0,  6, 13, 20, 27, 33, 40]), np.array([ 0,  7, 13, 20, 30, 39, 46, 53, 60, 70, 79])])

In [121]:
zzz

array([array([ 0,  4,  7, 11, 19, 26, 33, 39, 46, 53]),
       array([ 0,  6, 13, 20, 27, 33, 40]),
       array([ 0,  7, 13, 20, 30, 39, 46, 53, 60, 70, 79])], dtype=object)

In [12]:
train_mols = np.arange(all_all['all_xyz']['train'].shape[0])
val_mols = np.arange(all_all['all_xyz']['validate'].shape[0])


In [21]:
import MyNet2 as MyNet
#from MyNet import expand_features, expand_matrix, generator, myNet
reload (MyNet)

h_h = 64
#gen = MyNet.generator(train_mols, batch_size=128)
#valgen = MyNet.generator(val_mols, batch_size=128)

net = MyNet.myNet(65,18, h_h=h_h, h_g=24, nCells=6)
net.to(device)
poolnet = {2:  MyNet.poolNet(h_in=2*h_h + 29, h_h=h_h, nHidLayers=2).to(device),
          3:   MyNet.poolNet(h_in=3*h_h + 58, h_h=h_h, nHidLayers=2).to(device),
          4:   MyNet.poolNet(h_in=4*h_h + 88, h_h=h_h, nHidLayers=2).to(device)}

optimizer = Adam(list(net.parameters()) + list(poolnet[2].parameters())
                 + list(poolnet[3].parameters()) + list(poolnet[4].parameters()), lr=0.003)  

In [22]:
n_epochs = 500
train_loss = []
val_loss = []
batch_size = 128

In [39]:
def lfactor(epoch):
    if epoch < 78:
        return 1.
    if epoch < 137:
        return 0.3
    if epoch < 161:
        return 0.027
    if epoch < 181:
        return 0.0081
    if epoch < 201:
        return 0.000243
    else:
        return 0.00002187
    

In [40]:
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.3, patience=10, verbose=True, min_lr=1e-7)

scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lfactor)

In [44]:
scheduler.step(190)

In [45]:
optimizer.param_groups[0]

{'params': [Parameter containing:
  tensor([[-0.2186, -0.0832, -0.0468,  ..., -0.0521, -0.0619,  0.0640],
          [-0.1303, -0.1411,  0.3657,  ..., -0.5572,  0.5141,  0.0719],
          [-0.0685, -0.0424,  0.0759,  ..., -0.0663,  0.0464,  0.0580],
          ...,
          [-0.0227, -0.1402,  0.0653,  ..., -0.0601,  0.0172, -0.0950],
          [ 0.0501, -0.0055,  0.0950,  ..., -0.0958, -0.0803, -0.1206],
          [ 0.0726,  0.1440,  0.3242,  ...,  0.4253,  0.1094,  0.0213]],
         device='cuda:0', requires_grad=True), Parameter containing:
  tensor([-0.0449,  0.0665, -0.1560, -0.0222, -0.0347, -0.1231,  0.0794, -0.0599,
          -0.0506, -0.0107, -0.0353,  0.0269, -0.1006, -0.1639, -0.0926, -0.0856,
          -0.0324, -0.0921,  0.0321, -0.0398, -0.1471, -0.0004,  0.0555,  0.0118,
          -0.1447,  0.1167,  0.0046,  0.0017, -0.0760, -0.0429,  0.0887, -0.0565,
           0.0685,  0.0031,  0.1001,  0.0072, -0.0851, -0.1418,  0.0157,  0.0337,
          -0.0273, -0.0658, -0.1341,  0

In [ ]:
for epoch in range(168, n_epochs):
    gc.collect()
    print ("epoch = {}".format(epoch))
    for status in ['train', 'validate']:
        if (status == 'train'):
            gen = MyNet.generator(train_mols, batch_size=batch_size, seed=epoch)
            net.train()
            poolnet[2].train()
            poolnet[3].train()
            poolnet[4].train()
        else:
            gen = MyNet.generator(val_mols, batch_size=batch_size, seed=123456-epoch)
            net.train()
            poolnet[2].train()
            poolnet[3].train()
            poolnet[4].train()
            
        loss_per_batch = []
        ii = 0
        for i in tqdm.tqdm(gen, mininterval=10, maxinterval=200):
            ii+=1
            if (ii % 50) == 0:
                gc.collect()
            #time_now = time.time()
            #strucs_loc_i = strucs.loc[i]
            #df_graph_loc_i = df_graph.loc[i]
            #features_loc_i = features.loc[i]
            #trantest_loc_i = trantest.loc[i]
            #print ("time to loc_i {}".format(time.time() - time_now))
            #time_now = time.time()
            #A = df_graph_loc_i.AdjMatrix
            #A = np.stack(A.apply(MyNet.expand_matrix).values)
            #e_features = []

            #for col in ['EdgeLen', 'Multiplicity']:
            #    E = df_graph_loc_i[col]
            #    E = np.stack(E.apply(MyNet.expand_matrix).values)
            #    e_features.append(E)

            #E = df_graph_loc_i['EdgesOneHot']
            #e_3dfeatures = np.stack(E.apply(MyNet.expand_matrix, args=(False,)).values)

            #e_features = np.stack(e_features,axis=3)
            #e_features = np.concatenate((e_features, e_3dfeatures),axis=3)
            #fcounts = np.unique(features_loc_i.index.get_level_values(0), return_counts=True)[1]
            #ff = MyNet.expand_features(features_loc_i.values, fcounts)
            #ss = MyNet.expand_features(strucs_nn_features.loc[i].values, fcounts)
            #ff = np.concatenate((ff,ss),axis=2)

            #xyz = MyNet.expand_features(strucs[['x','y','z']].loc[i].values, fcounts)
            
            #xyz = torch.FloatTensor(xyz).to(device)
            #all_xyz[status].append(xyz)
            xyz = all_all['all_xyz'][status][i].to(device)
            
            #A = torch.FloatTensor(A).to(device)
            #all_A[status].append(A)
            
            A = all_all['all_A'][status][i].to(device)
            
            #e_features = torch.FloatTensor(e_features).to(device)
            #all_e_features[status].append(e_features)
            e_features = all_all['all_e_features'][status][i].to(device)
            
            #ff = torch.FloatTensor(ff).to(device)
            #all_ff[status].append(ff)
            ff = all_all['all_ff'][status][i].to(device)

            output = net(A, ff, e_features)
            
            
            #tt = trantest_loc_i.path
            #ytrue = trantest_loc_i.scalar_coupling_constant
            #batch_type = trantest_loc_i.type
            #batch_weights = trantest_loc_i.type.map(weights)
            #print ("time to prepare data {}".format(time.time() - time_now))
            #time_now = time.time()
            
            #all_tt[status].append(tt.values)
            #all_ytrue[status].append(torch.FloatTensor(ytrue))
            #all_batch_type[status].append(batch_type.values)
            #all_batch_weights[status].append(torch.FloatTensor(batch_weights.values))
            tt = all_all['all_tt'][status][i]
            ytrue= all_all['all_ytrue'][status][i]
            batch_type = all_all['all_batch_type'][status][i]
            batch_weights= all_all['all_batch_weights'][status][i]
            
            
            #print ("time to forward first nn {}".format(time.time() - time_now))
            #time_now = time.time()

            #print('a')
            #mol_to_ind = {mol:j for j,mol in enumerate(i)}
            loss = torch.FloatTensor([0.]).to(device).squeeze()
            
            
            for LLen in [2,3,4]:
                #indices_for_LLen[status][LLen].append(np.where((tt.apply(lambda x: len(x)) == LLen).values))
                index_for_LLen = all_all['indices_for_LLen'][status][LLen][i]
                #print (index_for_LLen)
                #print (tt)
                #break
                mol_num = np.concatenate([np.repeat(x, len(index_for_LLen[x][0])) for x in range(len(index_for_LLen))])
                query_num = np.concatenate([x[0] for x in index_for_LLen])
                if (len(query_num) == 0):
                    continue
                #tt2 = tt[tt.apply(lambda x: len(x)) == LLen]
                #if (tt2.shape[0] == 0):
                #if (index_for_LLen[0].shape[0] == 0):
                #    continue
                #tensor_weights = torch.FloatTensor(batch_weights[tt2.index].values).view(-1,1).to(device)
                #tensor_ytrue = torch.FloatTensor(ytrue[tt2.index].values).view(-1,1).to(device)
                tensor_weights = torch.stack([x[y] for x,y in zip(batch_weights[mol_num], query_num)]).to(device)
                tensor_ytrue = torch.stack([x[y] for x,y in zip(ytrue[mol_num], query_num)]).to(device)
                tt2 = np.array( [x[y] for x,y in zip(tt[mol_num], query_num)] , dtype=object)

                list_of_indices = {}
                tmptensors = []
                #print('b')
                for j in range(LLen):
                    #list_of_indices[j] = torch.LongTensor([[mol_to_ind[a[0]] for a in tt2.index],
                    #                                        [pathh[j] for pathh in tt2]]
                    #                                        ).to(device)
                    list_of_indices[j] = torch.LongTensor([mol_num,  [pathh[j] for pathh in tt2]]).to(device)
                    
                    
                    #print('c')
                    tmptensors.append(output[list_of_indices[j][0], list_of_indices[j][1]])
                    if (j == 0):
                        tmptensors.append(ff[list_of_indices[j][0], list_of_indices[j][1]][:,3:4]) #for first Hydrogen - always the same
                    else:
                        tmptensors.append(ff[list_of_indices[j][0], list_of_indices[j][1]][:,:9])
                    #print('d')
                for j in range(LLen):
                    for k in range(j + 1, LLen):
                        deltaR = torch.pow(torch.pow(xyz[list_of_indices[j][0], list_of_indices[j][1]] 
                                                     - xyz[list_of_indices[k][0], list_of_indices[k][1]], 2).sum(dim = -1, keepdim=True), 0.5)
                        tmptensors.append(deltaR)
                for j in range(LLen - 1):
                    k = j + 1
                    tmptensors.append(e_features[list_of_indices[j][0],list_of_indices[j][1], list_of_indices[k][1]])

                pulledtensor = torch.cat(tmptensors, dim=1).to(device)
                
                #print ("time to prepare queries {}".format(time.time() - time_now))
                #time_now = time.time()
                
                answ = poolnet[LLen](pulledtensor)                
                loss = loss + torch.sum(torch.abs((answ.view(-1) - tensor_ytrue)*tensor_weights))
                #thisind = tt2.index.get_level_values(1)
                #real_answ.loc[thisind].predicted_answ = answ.squeeze().detach().numpy()
                #if (status == 'validate'):
                #    real_answ.loc[thisind].is_validation = True
                            
                #print ("time to forward run pooling head {}".format(time.time() - time_now))
                #time_now = time.time()
            
            if (True):
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                                
                #print ("time to backprop {}".format(time.time() - time_now))
                #time_now = time.time()

            loss_per_batch.append(loss.item())
        #break    
        if (True):
            train_loss.append(np.mean(loss_per_batch))
        else:
            val_loss.append(np.mean(loss_per_batch))
        
    if len(train_loss):
        print("train loss epoch {} = {}".format(epoch, train_loss[-1]))
    if len(val_loss):
        print("val loss epoch {} = {}".format(epoch, val_loss[-1]))
    #scheduler.step(epoch)
    torch.save({
            'epoch': epoch,
            'net_state_dict': net.state_dict(),
            'poolnet_2_state_dict' : poolnet[2].state_dict(),
            'poolnet_3_state_dict' : poolnet[3].state_dict(),
            'poolnet_4_state_dict' : poolnet[4].state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': train_loss[-1],
            }, '/home/ikari/savednn/2608-full/epoch-{}.pt'.format(epoch))
    print("epoch {} train loss = {}   lr = {}".format(epoch, train_loss[-1], optimizer.param_groups[0]['lr']), 
                  file=open("/home/ikari/savednn/2608-full/log.txt", 'a'))






0it [00:00, ?it/s]

epoch = 168







50it [00:12,  3.86it/s]




100it [00:25,  3.87it/s]




150it [00:38,  3.87it/s]




200it [00:51,  3.87it/s]




250it [01:04,  3.87it/s]




300it [01:17,  3.87it/s]




350it [01:30,  3.86it/s]




400it [01:43,  3.86it/s]




450it [01:56,  3.86it/s]




500it [02:09,  3.85it/s]




550it [02:22,  3.85it/s]




598it [02:31,  3.95it/s]




0it [00:00, ?it/s]




50it [00:12,  3.88it/s]




67it [00:15,  4.23it/s]






train loss epoch 168 = 150.92879918795913
epoch = 169


0it [00:00, ?it/s]




50it [00:12,  3.86it/s]




100it [00:25,  3.87it/s]




150it [00:38,  3.85it/s]




200it [00:52,  3.84it/s]




250it [01:04,  3.85it/s]




300it [01:17,  3.85it/s]




350it [01:30,  3.85it/s]




400it [01:44,  3.84it/s]




450it [01:57,  3.85it/s]




500it [02:09,  3.85it/s]

In [156]:
answ.shape

torch.Size([1367, 1])

In [158]:
tensor_weights.shape

torch.Size([1367])

In [28]:
import pickle

In [ ]:
#torch.cuda.empty_cache()
#all_xyz = {'train':[], 'validate':[]}
#all_ytrue = {'train':[], 'validate':[]}
#all_e_features = {'train':[], 'validate':[]}
#all_A = {'train':[], 'validate':[]}

#all_ytrue = {'train':[], 'validate':[]}
#all_batch_type = {'train':[], 'validate':[]}
#all_batch_weights = {'train':[], 'validate':[]}
#all_tt = {'train':[], 'validate':[]}
#all_ff =  {'train':[], 'validate':[]}

#all_tensor_weights = {'train':{2:[],3:[],4:[]}, 'validate':[]}
#indices_for_LLen = {'train':{2:[],3:[],4:[]}, 'validate':{2:[],3:[],4:[]}}

In [30]:

all_all = {
    'all_xyz':all_xyz,
    'all_ytrue':all_ytrue,
    'all_e_features':all_e_features,
    'all_A':all_A,
    'all_batch_type':all_batch_type,
    'all_batch_weights':all_batch_weights,
    'all_tt':all_tt,
    'all_ff':all_ff,
    'indices_for_LLen':indices_for_LLen
}

In [32]:
with open ('/home/ikari/savednn/all_all.pkl', 'wb') as f:
    pickle.dump(all_all, f)

In [55]:
with open ('/home/ikari/savednn/all_all.pkl', 'rb') as f:
    all_all = pickle.load(f)

In [49]:
len(all_ff['train'])

76502

In [56]:
all_all['all_xyz']['train']=torch.cat(all_all['all_xyz']['train'])
all_all['all_xyz']['validate']=torch.cat(all_all['all_xyz']['validate'])


In [57]:
all_all['all_e_features']['train']=torch.cat(all_all['all_e_features']['train'])
all_all['all_e_features']['validate']=torch.cat(all_all['all_e_features']['validate'])

all_all['all_A']['train']=torch.cat(all_all['all_A']['train'])
all_all['all_A']['validate']=torch.cat(all_all['all_A']['validate'])



In [98]:
all_all['all_ff']['train']=torch.cat(all_all['all_ff']['train'])
all_all['all_ff']['validate']=torch.cat(all_all['all_ff']['validate'])

In [70]:
all_all['all_ytrue']['train']=np.array(all_all['all_ytrue']['train'],dtype=object)
all_all['all_ytrue']['validate']=np.array(all_all['all_ytrue']['validate'],dtype=object)

In [86]:
all_all['all_batch_type']['train']=np.array(all_all['all_batch_type']['train'],dtype=object)
all_all['all_batch_type']['validate']=np.array(all_all['all_batch_type']['validate'],dtype=object)

all_all['all_batch_weights']['train']=np.array(all_all['all_batch_weights']['train'],dtype=object)
all_all['all_batch_weights']['validate']=np.array(all_all['all_batch_weights']['validate'],dtype=object)



In [95]:
all_all['all_tt']['train']=np.array(all_all['all_tt']['train'],dtype=object)
all_all['all_tt']['validate']=np.array(all_all['all_tt']['validate'],dtype=object)

In [117]:
for LLen in [2,3,4]:
    
    all_all['indices_for_LLen']['train'][LLen] = np.array(all_all['indices_for_LLen']['train'][LLen], dtype=object)
    all_all['indices_for_LLen']['validate'][LLen] = np.array(all_all['indices_for_LLen']['validate'][LLen], dtype=object)

In [64]:
zzz = np.array(all_all['all_ytrue']['train'], dtype=object)

In [69]:
zzz[3]

tensor([-0.4562, -0.7096, -1.0554, -0.7803,  1.5382, -0.2940, -0.6519, -0.6613,
        -0.4950, -0.3121,  1.2666, -1.0321, -0.2121, -0.6767, -0.3812,  1.2287,
        -0.8459, -0.6378, -0.2061, -0.0879, -0.6810, -0.0685, -0.7181,  0.3568,
        -1.1214, -0.1086,  1.0168, -0.3896, -1.1804,  0.0276, -0.5041,  0.3086,
        -0.5991,  0.2909,  0.6498, -0.3417,  0.5862, -1.1002,  0.2823, -0.5105,
         0.0505,  1.6125, -1.1199, -0.1112, -0.6487,  0.3611, -0.2901, -0.5484,
         0.3201,  0.7690, -1.3939,  1.6882,  0.2730, -0.4692, -0.3018, -1.0498,
        -1.2553, -0.7838,  0.2814, -0.4779, -0.0689, -0.0820,  1.5038, -1.0599,
         0.7375,  0.4481,  1.5426, -0.2046, -0.6322, -0.2537, -0.4156,  0.4549,
        -1.1563,  0.2858,  0.3134, -0.6041, -0.2685,  0.3019,  0.5751, -0.2095])

In [124]:
with open ('/home/ikari/savednn/all_all_np_protocol4.pkl', 'wb') as f:
    pickle.dump(all_all, f, protocol=4)

In [ ]:
pulledtensor.shape

In [ ]:
abc = torch.FloatTensor([0]).squeeze().to(device)

In [ ]:
'/home/ikari/savednn/epoch-{}.pt'.format(2)

In [ ]:
abc.backward()

In [ ]:
loss

In [ ]:
%%time
batch_weights[tt2.index]

In [25]:
tt2

NameError: name 'tt2' is not defined

In [6]:
arrr = np.array([[2,3,4], [5,6,7]])

In [85]:
np.array([torch.FloatTensor(np.array([1,2,3])), torch.FloatTensor(np.array([4,5]))], dtype=object)

array([tensor([1., 2., 3.]), tensor([4., 5.])], dtype=object)

In [10]:
arrr[np.where(arrr>3)]

array([4, 5, 6, 7])

In [27]:
all_ff

{'train': [tensor([[[ 0.2211,  1.0000, -0.1873,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.2211,  1.0000, -0.1876,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.2211,  0.7500, -0.0260,  ...,  0.0000,  0.0000,  0.0000],
           ...,
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]]),
  tensor([[[ 2.2109e-01,  1.0000e+00, -1.8373e-01,  ...,  0.0000e+00,
             1.4305e+00,  0.0000e+00],
           [ 2.6541e+00,  5.0000e-01, -8.2222e-02,  ...,  0.0000e+00,
             0.0000e+00,  0.0000e+00],
           [ 2.2109e-01,  7.5000e-01, -2.3380e-05,  ...,  0.0000e+00,
             1.3283e+00,  1.3517e+00],
           ...,
           [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
             0.0000e+00,  0.0000e+00],
           [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            

In [40]:
tmptensors[1]

tensor([[0.1074],
        [0.1010],
        [0.1114],
        [0.1078],
        [0.0872],
        [0.1167],
        [0.0938],
        [0.1037],
        [0.0834],
        [0.0834]])

In [42]:
list_of_indices[j][1]

tensor([ 8,  9, 10, 11, 12, 13, 14, 15, 16, 17])

In [44]:
output.shape

torch.Size([29, 64])

In [164]:
torch.__version__

'1.0.1'



126it [03:40,  1.75s/it]